In [115]:
import pandas as pd
import json
import requests
import datetime
from collections import defaultdict


## the detailed api

In [116]:
URL = 'https://api.track.toggl.com/reports/api/v2/details?workspace_id=4482767&since=2020-08-1&until=2020-10-1&user_agent=aryaatharva18@gmail.com'
username = '7f45ea45a1833ac7e127047a738e4264' # also the token
password = 'api_token'

cols = [
#  'id',
#  'pid',
#  'tid',
#  'uid',
 'description',
 'start',
 'end',
 'updated',
 'dur',
#  'user',
#  'use_stop',
#  'client',
 'project',
#  'project_color',
 'project_hex_color'
#  'task',
#  'billable',
#  'is_billable',
#  'cur',
#  'tags'
]


df_raw = defaultdict(list)
ind = 1
data = []
data_in = [0]
while len(data_in) != 0:
    URL = URL + '&page=' + str(ind)
    response = requests.get(URL, auth = (username, password)).json()
    data_in = response['data']
    data += data_in
    for col in cols:
        for item in data_in:
            df_raw[col].append(item[col])
    ind += 1
df = pd.DataFrame.from_dict(df_raw)
df.head(4)

,description,start,end,updated,dur,project,project_hex_color
0,ED Assignment,2020-10-01T21:00:00+05:30,2020-10-01T22:00:00+05:30,2020-10-01T22:08:44+05:30,3600000,College Work,#525266
1,Hands on ML,2020-10-01T16:00:00+05:30,2020-10-01T18:00:00+05:30,2020-10-01T20:33:23+05:30,7200000,Books and Courses,#d92b2b
2,Hands on ML,2020-10-01T13:40:00+05:30,2020-10-01T14:40:00+05:30,2020-10-01T14:59:40+05:30,3600000,Books and Courses,#d92b2b
3,Hands on ML,2020-10-01T09:55:00+05:30,2020-10-01T11:55:00+05:30,2020-10-01T13:21:48+05:30,7200000,Books and Courses,#d92b2b


In [117]:
str(datetime.datetime.strptime(df['start'].tolist()[3].split('+')[0], '%Y-%m-%dT%H:%M:%S').date())

&#39;2020-10-01&#39;

In [118]:
df['start'].tolist()[3].split('+')[0]

&#39;2020-10-01T09:55:00&#39;

In [119]:
cols = [
        'description',
        'start',
        'end',
        'updated',
        'dur',
        'project',
        'project_hex_color'
        ]
print(cols)

[&#39;description&#39;, &#39;start&#39;, &#39;end&#39;, &#39;updated&#39;, &#39;dur&#39;, &#39;project&#39;, &#39;project_hex_color&#39;]


In [120]:
URL = 'https://api.track.toggl.com/reports/api/v2/summary?workspace_id=4482767&since=2020-08-1&until=2020-10-1&user_agent=aryaatharva18@gmail.com'
username = '7f45ea45a1833ac7e127047a738e4264' # also the token
password = 'api_token'
response = requests.get(URL, auth = (username, password)).json()


In [121]:
df_tasks = defaultdict(list)
df_projects = defaultdict(list)
df = defaultdict(list)

In [122]:
data = response['data'] # is a list
for project in data:
    tasks = project['items'] # is a list too, containing dictionaries 
    for task in tasks:
        df['task'].append(task['title']['time_entry'])
        df['task time duration'].append(task['time'])
        df['project'].append(project['title']['project'])
        # df['project name'].append(project['title']['project'])
        df['project time duration'].append(project['time'])
        df['project color'].append(project['title']['hex_color'])


In [123]:
df = pd.DataFrame.from_dict(df)
pd.set_option('display.max_rows', 500)
df = df.dropna()
df.head()
# df.iloc[-1]

,task,task time duration,project,project time duration,project color
0,DLP Course 1,694000,Books and Courses,269730117,#d92b2b
1,DLS Course 2 Week 2,12879000,Books and Courses,269730117,#d92b2b
2,DLS Course 2 Week 3,18050000,Books and Courses,269730117,#d92b2b
3,DLS Course 4 Week 1,11495000,Books and Courses,269730117,#d92b2b
4,DLS Course 4 Week 2,17518117,Books and Courses,269730117,#d92b2b


In [85]:
def build_sunburst_data(df, parents, labels,values, string = 'total'):
    
    main_parents = list(set(parents))
    main_values = []
    
    for main_parent in  main_parents:
        tmp = []
        for parent, label, value in zip(parents, labels, values):
            if parent == main_parent:
                tmp.append(value)
        main_values.append(sum(tmp))
    
    parents += [string] * len(main_parents)
    labels += main_parents
    values += main_values

    parents.append("")
    labels.append(string)
    values.append(0.5*sum(values))

    return parents, labels, values

In [86]:
parents, labels, values = build_sunburst_data(df, df['project'].tolist(), df['task'].tolist(),df['task time duration'].tolist(), string = 'total')

In [94]:
import plotly.graph_objects as go
# px.sunburst(df, path=['project', 'task'], values='task time duration')
fig = go.Figure()
fig.add_trace(
    go.Sunburst(
        values = values,
        parents = parents,
        labels = labels,
        branchvalues = 'total', 
        # maxdepth = 2,
        textinfo = "none"
    )
)
fig.show()

In [14]:
df_projects = pd.DataFrame.from_dict(df_projects)
df_projects.head()

,project name,project time duration,project color
0,Books and Courses,269730117,#d92b2b
1,chores,12483000,#525266
2,College Work,162017000,#525266
3,Googling/Browsing,10759000,#06a893
4,Kaggle,194288000,#e36a00


In [16]:
df_tasks = pd.DataFrame.from_dict(df_tasks)
df_tasks.head()

,task,task time duration,project
0,DLP Course 1,694000,Books and Courses
1,DLS Course 2 Week 2,12879000,Books and Courses
2,DLS Course 2 Week 3,18050000,Books and Courses
3,DLS Course 4 Week 1,11495000,Books and Courses
4,DLS Course 4 Week 2,17518117,Books and Courses
